In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Importing required libraries

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Approach
1. Load data
2. Select useful features
3. Extract numerical and categorical columns
4. Make pipeline
5. Do Hyperparameter tuning with cross-validation
- a. One with Ordinal Encoding
- b. One with OneHot Encoding, use best
6. Check MAE
7. Find best parameters
8. Train model
9. Predicting `survived` on `test_data`
10. Submit predictions

## 1. Load data

In [3]:
titanic_data = pd.read_csv("../input/titanic/train.csv", index_col="PassengerId")
test_data = pd.read_csv("../input/titanic/test.csv", index_col="PassengerId")

titanic_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Selecting useful features

In [4]:
useful_features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
X = titanic_data[useful_features]
Y = titanic_data["Survived"]
test_data = test_data[useful_features]

## 3. Extract numerical and categorical columns

In [5]:
num_cols = X.select_dtypes(exclude="object").columns
cat_cols = X.select_dtypes("object").columns
num_cols, cat_cols

(Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'),
 Index(['Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object'))

## 4. Make Pipeline

In [6]:
class CreatePipeline:
    """Create Pipeline
    methods:
        pipeline: Create Final Pipeline
        
        create_model: Create the provided model
        
        numerical_transformer: Transform numerical cols
        
        categorical_transformer: Transform categorical cols \
        OneHotEncoding / OrdinalEncoding
        
        data_preprocessor: Preprocess the data using ColumnTransformer     
        """
    
    def pipeline(self, *, preprocessor, model, verbose=False):
        """Creates pipeline
        params:
            preprocessor
            model
        """
        steps = [("preprocessor", preprocessor),
                 ("model", model)]
        return Pipeline(steps=steps, verbose=verbose)
    
    
    def numerical_transformer(self, *, strategy="mean", **params):
        """Transform numerical columns using `SimpleImputer`.
        params:
            strategy: "mean" | "median" | "most_frequent" | "constant"
            **params: extra keyword args for SimpleImputer"""
        
        transformer = SimpleImputer(strategy=strategy, **params)
        return transformer

    
    def categorical_transformer(self, *, 
                                imp_strategy="most_frequent", 
                                encoder_type="Ordinal", 
                                imp_params={}, encoder_params={}):
        """Transform categorical columns by making Pipeline
        `SimpleImputer` | `OneHotEncoder` | `OrdinalEncoder`.
        args:
            imp_strategy: strategy for imputer values can be
                "most_frequent" | "constant"
            encoder_type: encoder type,
                "Ordinal" | "OneHot"
        kwargs:
            imp_params: keyword args for `SimpleImputer`.
            encoder_params: keyword args for encoder.`
        """
        if not encoder_type in ("Ordinal", "OneHot"):
            raise ValueError(f"Inappropriate value for encoder_type passed: {encoder_type}\
            Takes one of 'Ordinal' | 'OneHot'.")
        
        encoder = OrdinalEncoder if encoder_type=="Ordinal" else OneHotEncoder
        transformer = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy=imp_strategy, **imp_params)),
            (encoder_type, encoder(**encoder_params))
        ])
        return transformer
    
    
    def data_preprocessor(self, *, transformers):
        """Preprocess the data using `ColumnTransformer`.
        Pass extact list of transformers
        to be passed in `ColumnTransformer`.
        each tuple consist of: (transformer_name,
                                transformer,
                                list_of_columns)."""
        preprocessor = ColumnTransformer(transformers=transformers)
        return preprocessor
    
    
    def create_model(self, *, model, random_state=0, n_estimators=1000, **kwargs):
        """Creates the model.
        **kwargs: keyword args for model."""
        my_model = model(random_state=random_state, n_estimators=n_estimators, **kwargs)
        return my_model

In [7]:
cp = CreatePipeline()
num_transformer = cp.numerical_transformer()
cat_oh_transformer = cp.categorical_transformer(encoder_type="OneHot", encoder_params={"handle_unknown": "ignore"})
cat_ord_transformer = cp.categorical_transformer(encoder_type="Ordinal", encoder_params={"handle_unknown":"use_encoded_value", "unknown_value":-1})
oh_preprocessor = cp.data_preprocessor(
                    transformers=[("num", num_transformer, num_cols),
                                  ("cat", cat_oh_transformer, cat_cols)
                                 ])
ord_preprocessor = cp.data_preprocessor(
                    transformers=[("num", num_transformer, num_cols),
                                  ("cat", cat_ord_transformer, cat_cols)
                                 ])

## 5. Hyperparamter tuning with Cross-validation
### a. with `OneHot Encoding`

In [8]:
n_estimators = [100, 250, 500, 750]
max_depths = [5, 10, 20]
oh_maes = {}
i = 0
for n in n_estimators:
    for md in max_depths:
        i += 1
        model = cp.create_model(model=RandomForestClassifier, n_estimators=n, max_depth=md)
        pipeline = cp.pipeline(preprocessor=oh_preprocessor, model=model)
        scores = -1 * cross_val_score(pipeline, X, Y, cv=10, verbose=True,
                                scoring="neg_mean_absolute_error")
        mae = scores.mean()
        oh_maes[i] = [n, md, mae]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    6.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

### b. with `Ordinal Encoding`

In [9]:
n_estimators = [100, 250, 500, 750]
max_depths = [5, 10, 20]
ord_maes = {}
i = 0
for n in n_estimators:
    for md in max_depths:
        i += 1
        model = cp.create_model(model=RandomForestClassifier, n_estimators=n, max_depth=md)
        pipeline = cp.pipeline(preprocessor=ord_preprocessor, model=model)
        scores = -1 * cross_val_score(pipeline, X, Y, cv=10, verbose=True,
                                scoring="neg_mean_absolute_error")
        mae = scores.mean()
        ord_maes[i] = [n, md, mae]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    6.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

## 6. Checking MAEs
### a. `OneHot Encoding`

In [10]:
for i in oh_maes:
    n, md, mae = oh_maes[i]
    print(f"{i}.\tN_estimators: {n}\tmax_depth: {md}\tMAE: {mae}")

1.	N_estimators: 100	max_depth: 5	MAE: 0.23676654182272155
2.	N_estimators: 100	max_depth: 10	MAE: 0.19410736579275906
3.	N_estimators: 100	max_depth: 20	MAE: 0.17283395755305866
4.	N_estimators: 250	max_depth: 5	MAE: 0.23901373283395758
5.	N_estimators: 250	max_depth: 10	MAE: 0.19749063670411984
6.	N_estimators: 250	max_depth: 20	MAE: 0.1705992509363296
7.	N_estimators: 500	max_depth: 5	MAE: 0.23227215980024968
8.	N_estimators: 500	max_depth: 10	MAE: 0.18965043695380773
9.	N_estimators: 500	max_depth: 20	MAE: 0.1683395755305868
10.	N_estimators: 750	max_depth: 5	MAE: 0.22891385767790257
11.	N_estimators: 750	max_depth: 10	MAE: 0.18965043695380773
12.	N_estimators: 750	max_depth: 20	MAE: 0.1728339575530587


### b. `Ordinal Encoding`

In [11]:
for i in ord_maes:
    n, md, mae = ord_maes[i]
    print(f"{i}.\tN_estimators: {n}\tmax_depth: {md}\tMAE: {mae}")

1.	N_estimators: 100	max_depth: 5	MAE: 0.17843945068664166
2.	N_estimators: 100	max_depth: 10	MAE: 0.18067415730337077
3.	N_estimators: 100	max_depth: 20	MAE: 0.17506866416978778
4.	N_estimators: 250	max_depth: 5	MAE: 0.17394506866416976
5.	N_estimators: 250	max_depth: 10	MAE: 0.17280898876404496
6.	N_estimators: 250	max_depth: 20	MAE: 0.17730337078651687
7.	N_estimators: 500	max_depth: 5	MAE: 0.17394506866416976
8.	N_estimators: 500	max_depth: 10	MAE: 0.17393258426966293
9.	N_estimators: 500	max_depth: 20	MAE: 0.1784269662921348
10.	N_estimators: 750	max_depth: 5	MAE: 0.1728214731585518
11.	N_estimators: 750	max_depth: 10	MAE: 0.1739325842696629
12.	N_estimators: 750	max_depth: 20	MAE: 0.1750561797752809


In [12]:
min(oh_maes, key=lambda x: oh_maes[x][2])

9

In [13]:
min(ord_maes, key=lambda x: ord_maes[x][2])

5

## 7. Best parameters:
- a. `Ordinal Encoding`
- - `N_estimators: 250` & `max_depth:10` & `MAE:0.17280898876404496`
- b. `One Hot Encoding`
- - `N_estimators: 500` &	`max_depth: 20` & `MAE: 0.1683395755305868`

### `OneHotEncoding` did better

In [14]:
# using onehot encoding
best_n_estimators = 500
best_max_depth = 20

## 8. Training model on best parameters with `OneHotEncoding` as preprocessor

In [15]:
model = cp.create_model(model=RandomForestClassifier, n_estimators=best_n_estimators, max_depth=best_max_depth)
pipeline = cp.pipeline(preprocessor=oh_preprocessor, model=model)
pipeline.fit(X, Y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object'))])),
                ('model',
                 RandomForestClassifier(max_depth=20, n_estimators=500,
                                        random_state=0))])

## 9. Predicting `survived` on `test_data`

In [16]:
test_preds = pipeline.predict(test_data)
test_preds

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [17]:
output = pd.DataFrame({"PassengerId": test_data.index, "Survived": test_preds})
output.to_csv("./submission_2.csv", index=False)